# Basics of mobile robotics project - The Thytanic Navigation (2024)

## 1. Introduction & Context

| **Contributors**         | **Sciper**  | **Role**             | **Studies**                                                             |
|---------------------------|-------------|-----------------------|---------------------------------------------------------------------------|
| **Alessio Desogus**       | 301705      | Local Navigation      | Bachelor and Master in Mechanical Engineering.                            |
|                           |             |                       |                                                                           |
|                           |             |                       |                                                                           |
|                           |             |                       |                                                                           |

Date of deliverable: 05.12.2024  

Our main sources were the exercise sessions of the course and ....

### Project Description




### Environment



In [ ]:
import cv2 as cv

from vision import Camera, get_arena
from controlling_thymio import ManipulatingThymio, State
from global_path import optimal_path, generate_points_on_path
from kalman import kalman_filter, Q